In [1]:
import pandas as pd
import matplotlib.pyplot as pls
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score, roc_auc_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [2]:
%matplotlib inline

In [3]:
path_to_data = r'C:\Users\Machine\Downloads\churn_daata\churn.csv'

In [4]:
df = pd.read_csv(path_to_data)

In [5]:
df.head(3)

,Unnamed: 0,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1


In [14]:
df[['days_since_last_login', 'avg_transaction_value', 'churn_risk_score' ]]

,days_since_last_login,avg_transaction_value,churn_risk_score
0,17,53005.25,0
1,16,12838.38,0
2,14,21027.00,1
3,11,25239.56,1
4,20,24483.66,1
...,...,...,...
36987,2,27277.68,1
36988,13,11069.71,1
36989,12,38127.56,1
36990,15,2378.86,0


In [6]:
result_dataset = df[['avg_transaction_value','days_since_last_login','churn_risk_score']]

In [7]:
result_dataset.head(5)

,avg_transaction_value,days_since_last_login,churn_risk_score
0,53005.25,17,0
1,12838.38,16,0
2,21027.00,14,1
3,25239.56,11,1
4,24483.66,20,1


In [8]:
result_dataset.rename(columns = {'avg_transaction_value':'Budget', 
                                 'days_since_last_login': 'days_inactive',
                                 'churn_risk_score': 'is_churned'}, inplace = True)

C:\Users\Machine\AppData\Local\Temp\ipykernel_17280\1706070767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_dataset.rename(columns = {'avg_transaction_value':'Budget',


In [9]:
result_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Budget         36992 non-null  float64
 1   days_inactive  36992 non-null  int64  
 2   is_churned     36992 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 867.1 KB


In [10]:
train_data, val_data = train_test_split(result_dataset, test_size=0.2, random_state=42)
train_data, test_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [11]:
x_train = train_data.drop(['is_churned'], axis=1)
x_val = val_data.drop(['is_churned'], axis=1)
x_test = test_data.drop(['is_churned'], axis=1)

In [12]:
y_train = train_data['is_churned']
y_val = val_data['is_churned']
y_test = test_data['is_churned']

In [14]:
class_negative = y_train.value_counts()[1]
class_positive = y_train.value_counts()[0]

In [17]:
y_train.value_counts()

1    12761
0    10913
Name: is_churned, dtype: int64

In [100]:
scaler = StandardScaler().fit(x_train)

In [113]:
x_train_sc = scaler.transform(x_train)


In [114]:
scaler = StandardScaler().fit(x_test)
x_test_sc = scaler.transform(x_train)

In [ ]:
scaler = StandardScaler().fit(x_test)
x_test_sc = scaler.transform(x_train)

In [196]:
model =  CatBoostClassifier(    iterations=10000,
                                learning_rate=0.001,
                                depth=2,
                                verbose=False,
                                class_weights=[1, (class_negative / class_positive)]                 
                                )

In [197]:
val_pool = Pool(x_val, y_val)

In [124]:
pipe = make_pipeline(StandardScaler(), model)

In [125]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x000002914B9DD280>)])

In [126]:
pipe.score(x_test, y_test)

0.6242608548741342

In [198]:
model.fit(x_train, y_train, plot=True, eval_set=val_pool, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [130]:
pred_class

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [131]:
 roc_auc_score(y_test, pred_class)

0.586741207988478

In [199]:
pred_class = model.predict_proba(x_test)
print('ROC-AUC score ' + str(roc_auc_score(y_test.to_list(), pred_class[:, 0])))

ROC-AUC score 0.4182968935103689


In [5]:
resulted_dataset = df[['avg_transaction_value','days_since_last_login','churn_risk_score',  
                      'membership_category','age','gender' ]]

In [6]:
resulted_dataset.rename(columns = {'churn_risk_score':'is_churned'}, inplace = True)

C:\Users\Machine\AppData\Local\Temp\ipykernel_6288\583456562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resulted_dataset.rename(columns = {'churn_risk_score':'is_churned'}, inplace = True)


In [7]:
resulted_dataset.columns

Index(['avg_transaction_value', 'days_since_last_login', 'is_churned',
       'membership_category', 'age', 'gender'],
      dtype='object')

In [8]:
resulted_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   avg_transaction_value  36992 non-null  float64
 1   days_since_last_login  36992 non-null  int64  
 2   is_churned             36992 non-null  int64  
 3   membership_category    36992 non-null  object 
 4   age                    36992 non-null  int64  
 5   gender                 36992 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 1.7+ MB


In [9]:
train_data, val_data = train_test_split(resulted_dataset, test_size=0.2, random_state=42)
train_data, test_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [10]:
x_train = train_data.drop(['is_churned'], axis=1)
x_val = val_data.drop(['is_churned'], axis=1)
x_test = test_data.drop(['is_churned'], axis=1)

In [11]:
y_train = train_data['is_churned']
y_val = val_data['is_churned']
y_test = test_data['is_churned']

In [12]:
class_negative = y_train.value_counts()[1]
class_positive = y_train.value_counts()[0]

In [13]:
model2 =  CatBoostClassifier(   cat_features = ('membership_category', 'gender'),
                                iterations=10000,
                                learning_rate=0.0001,
                                depth=2,
                                verbose=False,
                                class_weights=[1, (class_negative / class_positive)]                 
                                )

In [16]:
pipe = make_pipeline(StandardScaler(), model2)

In [17]:
val_pool = Pool(x_val, y_val, cat_features = ['membership_category', 'gender'],
                feature_names = ['avg_transaction_value', 'days_since_last_login', 'membership_category',
       'age', 'gender'])

In [18]:
train_pool = Pool(x_train, y_train, cat_features = ['membership_category', 'gender'],
                feature_names = ['avg_transaction_value', 'days_since_last_login', 'membership_category',
       'age', 'gender'])

In [19]:
test_pool = Pool(x_test, y_test, cat_features = ['membership_category', 'gender'],
                feature_names = ['avg_transaction_value', 'days_since_last_login', 'membership_category',
       'age', 'gender'])

In [20]:
pipe.fit(train_pool)

TypeError: float() argument must be a string or a number, not 'Pool'

In [178]:
model2.fit(x_train, y_train, plot=True, eval_set=val_pool, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [185]:
pred_class = model2.predict(test_pool)


In [186]:
pred_class

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [187]:
roc_auc_score(y_test, pred_class)

0.8540984871074291